Read data.

In [94]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.feature_selection import SelectKBest, f_regression, RFE
import numpy as np

data = pd.read_csv('./HW2 附件/HW2_bike-sharing_train.csv')

# Preprocessing the dataset.

In [96]:

def seperate_dt(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    # # Extract year, month, day, hour, minute, and second as separate columns
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    return df

# check nan
def check_nan(df):
    return df.isna().any().any()

# turn season into encoding
def onehot_encoding(df, column_name):
    df_encoded = pd.get_dummies(df, columns=[column_name], prefix=[column_name])
    print(df.shape, "->", df_encoded.shape)
    return df_encoded

def standardization(df):
    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Fit and transform the data using StandardScaler
    df_standardized = scaler.fit_transform(df)
    df_standardized = pd.DataFrame(df_standardized, columns=df.columns)
    return df_standardized

def pprint(output = '\n', show_time = False): # print and fprint at the same time
    global filename
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))

        f.write(str(output))
        f.write('\n')

def normalization(df):
    # Initialize the MinMaxScaler
    scaler = MinMaxScaler()
    # Fit and transform the data using MinMaxScaler
    df_normalized = scaler.fit_transform(df)
    df_normalized = pd.DataFrame(df_normalized, columns=df.columns)
    return df_normalized
# filename = "Linear_Regression.txt"
# pprint("This is a Test", True)
df = data.copy()
df = seperate_dt(df).drop(columns=['datetime'])
check_nan(df)
df = onehot_encoding(df,'season')
print(df)


(8386, 13) -> (8386, 16)
      holiday  workingday  weather   temp   atemp  humidity  windspeed  count  \
0           0           0        1   9.84  14.395        81     0.0000     16   
1           0           0        1   9.02  13.635        80     0.0000     40   
2           0           0        1   9.02  13.635        80     0.0000     32   
3           0           0        1   9.84  14.395        75     0.0000     13   
4           0           0        1   9.84  14.395        75     0.0000      1   
...       ...         ...      ...    ...     ...       ...        ...    ...   
8381        0           1        1  16.40  20.455        50    26.0027    562   
8382        0           1        1  15.58  19.695        50    23.9994    569   
8383        0           1        1  15.58  19.695        50    26.0027    336   
8384        0           1        1  14.76  17.425        57    15.0013    241   
8385        0           1        1  13.94  17.425        61     6.0032    129   

  

Seperate into X and y.

In [97]:
X = df.drop(columns=['count'])
y = df['count']
X.shape, y.shape

((8386, 15), (8386,))

# Split up dataset into Train, Val and Test.

In [98]:

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6708, 15) (6708,)
(839, 15) (839,)
(839, 15) (839,)


In [152]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
linear_regression = LinearRegression()
poly3 = PolynomialFeatures(degree=3)
poly4 = PolynomialFeatures(degree=4)
lasso_regression4 = make_pipeline(poly4,StandardScaler(), Lasso())
poly_regression3 = make_pipeline(poly3, linear_regression)

# Create a voting ensemble with the individual regressors
voting_regressor = VotingRegressor(estimators=[
    ('lasso0', lasso_regression4),
    ('lasso1', lasso_regression4),
    ('poly0', poly_regression3),
    ('poly1', poly_regression3),
    ('poly2', poly_regression3),
])

# Fit the ensemble model to the training data
voting_regressor.fit(X_train, y_train)

# Make predictions with the ensemble model
y_train_pred = voting_regressor.predict(X_train)
y_train_pred = abs(y_train_pred).astype(int)
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)
print("Training Set Metrics:")
print("MSE: ", mse_train)
print("R-squared: ", r2_train)

# Make predictions with the ensemble model
y_pred = voting_regressor.predict(X_val)
# Calculate the Mean Squared Error and R-squared
y_pred = abs(y_pred).astype(int)
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
print("Voting Regressor Metrics:")
print("Mean Squared Error:", mse)
print("R-squared:", r2)

/Users/liushiwen/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.707e+04, tolerance: 2.224e+04
  model = cd_fast.enet_coordinate_descent(
/Users/liushiwen/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.707e+04, tolerance: 2.224e+04
  model = cd_fast.enet_coordinate_descent(


Training Set Metrics:
MSE:  11073.946481812762
R-squared:  0.66603473723869
Voting Regressor Metrics:
Mean Squared Error: 10024.902264600714
R-squared: 0.6553502398509436


In [108]:
ape = sum(np.abs((y_val - y_pred) / y_val) * 100)
print(ape/ len(y_val))

245.52041700503884


In [156]:
# Base model
model = LinearRegression()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_train_pred = abs(y_train_pred).astype(int)

mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)
print("Training Set Metrics:")
print("MSE: ", mse_train)
print("R-squared: ", r2_train)

y_pred = model.predict(X_val)
y_pred = abs(y_pred).astype(int)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("Voting Regressor Metrics:")
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Training Set Metrics:
MSE:  19760.88908765653
R-squared:  0.4040561305407345
Voting Regressor Metrics:
Mean Squared Error: 16446.34684147795
R-squared: 0.43458506181562984


In [157]:
y_pred = voting_regressor.predict(X_test)
y_pred = abs(y_pred).astype(int)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Voting Regressor Metrics:")
print("Mean Squared Error:", mse)
print("R-squared:", r2)

y_pred = model.predict(X_test)
y_pred = abs(y_pred).astype(int)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Voting Regressor Metrics:")
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Voting Regressor Metrics:
Mean Squared Error: 12295.0
R-squared: 0.6166418276818644
Voting Regressor Metrics:
Mean Squared Error: 19309.891537544696
R-squared: 0.3979174682802302


In [158]:
test_data = pd.read_csv('./HW2 附件/HW2_bike-sharing_test.csv')

In [159]:
test_df = test_data.copy()
test_df = seperate_dt(test_df).drop(columns=['datetime'])
check_nan(test_df)
test_df = onehot_encoding(test_df,'season')
print(test_df)

(2500, 12) -> (2500, 15)
      holiday  workingday  weather   temp   atemp  humidity  windspeed  year  \
0           0           0        3  14.76  17.425        93     8.9981  2012   
1           0           0        1  10.66  12.880        41    15.0013  2011   
2           0           1        2  13.94  18.180        42     0.0000  2012   
3           1           0        1  33.62  37.120        43     6.0032  2011   
4           0           1        1  38.54  41.665        29    16.9979  2012   
...       ...         ...      ...    ...     ...       ...        ...   ...   
2495        0           1        3  18.04  21.970        77    15.0013  2011   
2496        0           1        1   9.02  11.365        60     8.9981  2011   
2497        0           0        3  17.22  21.210        67    16.9979  2011   
2498        0           0        3  17.22  21.210        88    16.9979  2011   
2499        0           0        1  26.24  30.305        65    11.0014  2012   

      month  d

In [160]:
X = test_df
X.shape

(2500, 15)

In [161]:
y_real_pred = voting_regressor.predict(X)
y_real_pred = abs(y_real_pred).astype(int)
y_real_pred.shape

(2500,)

In [162]:
result = y_real_pred.reshape(-1, 1)  # Reshape to (1000, 1)
# Create a DataFrame with one column 'left'
result_df = pd.DataFrame(data=result, columns=['count'])
print(result_df)
result_df.to_csv('HW2_bike-sharing_test_sol.csv', index=False)

      count
0       194
1        86
2       285
3       423
4       276
...     ...
2495    158
2496    102
2497    106
2498     55
2499    186

[2500 rows x 1 columns]
